In [1]:
import sys
from pathlib import Path
from os import getcwd

from datetime import date, timedelta

sys.path.append(str(Path(getcwd()).parent))
print(sys.path)

['c:\\SESA\\dev\\notebooks', 'c:\\Users\\est.joaosilva\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles\\vscode_datascience_helpers\\..\\.does-not-exist', 'c:\\Users\\est.joaosilva\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles', 'c:\\Users\\est.joaosilva\\.vscode\\extensions\\ms-toolsai.jupyter-2021.2.576440691\\pythonFiles\\lib\\python', 'C:\\ProgramData\\Anaconda3\\python38.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\est.joaosilva\\.ipython', 'c:\\SESA\\dev']


In [2]:
from os.path import join
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sys import exit

In [3]:
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.tb_pacientes import TbPacientes
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin import default_input, default_output


---------------------------------------------------------------

root: c:\SESA\dev\bulletin
parent: c:\SESA\dev
default_input: c:\SESA\dev\input
default_output: c:\SESA\dev\output

---------------------------------------------------------------



In [4]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
# municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']

In [5]:
# tb_pacientes = TbPacientes()
# tb_pacientes.update()
# casost = tb_pacientes.get()
casost = pd.read_pickle('tb_pacientes.pkl')#tb_pacientes.get()
casost['nome'] = casost['nome'].apply(lambda nome: ''.join(filter(lambda x: ('A' <= x <= 'Z') or ( x == ' '), nome)))
casost = casost.sort_values(['dt_notificacao','nome','idade'])
print(f"casost len: {len(casost)}\n")
casost.loc[casost['obito']=='Sim','obito'] = 'SIM'
casost.loc[casost['obito']!='SIM','obito'] = 'NAO'
# casost = casost.loc[casost['dt_notificacao'] <= '2021-05-27'].copy()
print(f"casost len: {len(casost)}\n")
casost.groupby('obito')[['sexo']].count()

casost len: 1156347

casost len: 1156347



,sexo
obito,
NAO,1127799
SIM,28548


In [6]:
casost.groupby('status')[['sexo']].count()

,sexo
status,
Recuperado,812456
recuperado,2


In [7]:
exclusoes = pd.read_pickle(join(default_output,'exclusoes.pkl'))
exclusoes = exclusoes.rename(columns={'identificacao':'id', 'hash':'hash_com_ibge'})
exclusoes.shape

(250, 31)

In [8]:
casost['hash_final'] = casost.apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid'])+date_hash(row['dt_notificacao']), axis=1)
exclusoes['hash_final'] = exclusoes.apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid'])+date_hash(row['comunicacao']), axis=1)

In [9]:
exclusoes = exclusoes.merge(how = 'left', on='hash_final', right = casost[['identificacao', 'hash_final']])
exclusoes = exclusoes.rename(columns={'identificacao':'id_hash1'})
exclusoes = exclusoes.drop_duplicates('hash_final')
exclusoes.shape

(250, 33)

In [10]:
exclusoes['hash'] = exclusoes.apply(lambda row: normalize_hash(row['nome'])+str(row['idade'])+normalize_hash(row['mun_resid']), axis=1)
exclusoes = exclusoes.merge(how = 'left', on='hash', right = casost[['identificacao', 'hash']])
exclusoes = exclusoes.rename(columns={'identificacao':'id_hash2'})
exclusoes.shape

(825, 35)

In [11]:
exclusoes.loc[exclusoes['id_hash1'].isnull(), 'identificacao'] = exclusoes.loc[exclusoes['id_hash1'].isnull(), 'id_hash2']
exclusoes.loc[exclusoes['identificacao'].isnull(), 'identificacao'] = exclusoes.loc[exclusoes['id_hash2'].isnull(), 'id_hash1']
exclusoes.loc[exclusoes['identificacao'].isnull(), 'identificacao'] = exclusoes.loc[exclusoes['identificacao'].isnull(), 'id_hash1']

In [12]:
exclusoes = exclusoes.drop_duplicates(['hash', 'hash_final'])

In [25]:
exclusoes.drop(index = exclusoes.loc[exclusoes['identificacao'].isnull()].index).shape

(250, 36)

In [26]:
#exclusoes = exclusoes.drop(index = exclusoes.loc[exclusoes['identificacao'].isnull()].index)

In [27]:
exclusoes[['identificacao']].to_excel(f"exclusoes_{date.today().day}_{date.today().month}.xlsx")